# **Logging In**

In [1]:
#getting hf tokens to perform operations with their API
from google.colab import userdata
readToken = userdata.get('readToken')
writeToken = userdata.get('writeToken')

from huggingface_hub import login

login(readToken)


# **Loading** **Datasets**

In [2]:
#dataset dependencies

!pip install --upgrade datasets
!pip install --upgrade huggingface_hub

# **Edit the repos list with new datasets for processing each time**

In [3]:
#needs to be changed when processing new datasets
from datasets import load_dataset
repos = ["GAIR/FRoG", "GAIR/LIMR", "GAIR/LIMO", "ise-uiuc/Magicoder-Evol-Instruct-110K", "Vezora/Open-Critic-GPT", "qwedsacf/grade-school-math-instructions"]

# **Loaders**

In [4]:
#splitLoader will be included in the model file too for specific split loading to train it

from datasets import get_dataset_split_names

def splitLoader(Repos, Split="train"): #loads specific splits (train split by default)... will be used in model training and testing
  Names = []
  dfs = []
  for dataset in Repos:
    try:
      dfs.append(load_dataset(dataset, split=Split))
      Names.append(i)
    except Exception as e:
      print(f"Exception received: {e}")
      inp = input("Enter a different split: ")
      availableSplits = get_dataset_split_names(dataset)
      splitLen = len(availableSplits)
      c = 0
      while (inp.strip() != "") and (inp in availableSplits) and (c < splitLen):
        dfs.append(load_dataset(dataset, split=inp))
        Names.append(dataset + "_" + inp)
        inp = input("Enter another split for the same dataset [do not enter anything otherwise]: ")
        c += 1
  return Names, dfs

def loader(Repos): #loads all splits - best for preprocessing the datasets as a whole
  Names = []
  DFs = []
  z = 0
  for i in Repos:
      DFs.append(load_dataset(i))
      Names.append(i)
      z += 1
  return Names, DFs




In [5]:
names, dfs = loader(repos)

for i in range(len(dfs)):
  print(f"dataset from {names[i]}: ")
  print(f"{dfs[i]} \n--------------------------------\n")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.60k [00:00<?, ?B/s]

easy-00000-of-00001.parquet:   0%|          | 0.00/718k [00:00<?, ?B/s]

hard-00000-of-00001.parquet:   0%|          | 0.00/717k [00:00<?, ?B/s]

Generating easy split:   0%|          | 0/2044 [00:00<?, ? examples/s]

...

Generating train split:   0%|          | 0/55117 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/852 [00:00<?, ?B/s]

(…)-00000-of-00001-3f5d416810641542.parquet:   0%|          | 0.00/2.55M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8792 [00:00<?, ? examples/s]

dataset from GAIR/FRoG: 
DatasetDict({
    easy: Dataset({
        features: ['id', 'question', 'target_percentage_mention', 'quantifier', 'quantifier_strength_gap', 'answer', 'origin_question', 'origin_reasoning', 'raw_question', 'source'],
        num_rows: 2044
    })
    hard: Dataset({
        features: ['id', 'question', 'target_percentage_mention', 'quantifier', 'quantifier_strength_gap', 'answer', 'origin_question', 'origin_reasoning', 'raw_question', 'source'],
        num_rows: 2044
    })
}) 
--------------------------------

dataset from GAIR/LIMR: 
DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'answer', 'source'],
        num_rows: 1389
    })
}) 
--------------------------------

dataset from GAIR/LIMO: 
DatasetDict({
    train: Dataset({
        features: ['question', 'solution', 'answer'],
        num_rows: 817
    })
}) 
--------------------------------

dataset from ise-uiuc/Magicoder-Evol-Instruct-110K: 
DatasetDict({
    train: Dataset({
    

# **Basic Preprocessing Utilities**

In [6]:
def countNulls(dataset): #across all splits
  counter = 0
  for split in dataset.keys():
    for row in dataset[split]:
      for column in row.keys():
        if(row[column] is None):
          counter+=1
  return counter

def dropColumns(dataset, columnNames=[]): #across all splits
  if(columnNames):
    for split in dataset.keys():
      for i in columnNames:
        if(i not in dataset[split].column_names):
          break
        dataset[split] = dataset[split].remove_columns(column_names = columnNames)
  return dataset

def renameAllColumns(dataset, newNames=[]): #across all splits
  defaultSplit = list(dataset.keys())[0]
  if (len(newNames) != len(dataset[defaultSplit].column_names)):
    return

  for i in list(zip(dataset[defaultSplit].column_names, newNames)):
    for split in dataset.keys():
      if(i[0] == i[1]):
        continue
      dataset[split] = dataset[split].rename_column(*i)

  return dataset


In [7]:

df = dfs[0]
#datasetInfo(df)

df.column_names

#df = df['question', 'origin_question', 'answer', 'origin_reasoning'] #valid for single-split datasets


df = dropColumns(df, ['id', 'target_percentage_mention', 'quantifier', 'quantifier_strength_gap', 'raw_question', 'source'])

df

new_names = ['instruction', 'response', 'original_instruction', 'reasoning_response']
df = renameAllColumns(df, new_names)
print(df)
countNulls(df)

login(writeToken)
df.push_to_hub("O047/prepSIFT_FuzzyMath", private=False)

DatasetDict({
    easy: Dataset({
        features: ['instruction', 'response', 'original_instruction', 'reasoning_response'],
        num_rows: 2044
    })
    hard: Dataset({
        features: ['instruction', 'response', 'original_instruction', 'reasoning_response'],
        num_rows: 2044
    })
})


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/O047/prepSIFT_FuzzyMath/commit/5ef2c96bcc642a9f70beeba6c2c3f744ee67fef8', commit_message='Upload dataset', commit_description='', oid='5ef2c96bcc642a9f70beeba6c2c3f744ee67fef8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/O047/prepSIFT_FuzzyMath', endpoint='https://huggingface.co', repo_type='dataset', repo_id='O047/prepSIFT_FuzzyMath'), pr_revision=None, pr_num=None)

In [8]:
df = dfs[1]
print(df.column_names)
df = dropColumns(df, ['id', 'source'])


df = renameAllColumns(df, ['instruction', 'response'])
print(df)
countNulls(df)
df.push_to_hub("O047/prepSIFT_MathReasoning1", private=False)

{'train': ['id', 'prompt', 'answer', 'source']}
DatasetDict({
    train: Dataset({
        features: ['instruction', 'response'],
        num_rows: 1389
    })
})


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/O047/prepSIFT_MathReasoning1/commit/c186f80da2fd04aefc848d35992b4b3986ad1620', commit_message='Upload dataset', commit_description='', oid='c186f80da2fd04aefc848d35992b4b3986ad1620', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/O047/prepSIFT_MathReasoning1', endpoint='https://huggingface.co', repo_type='dataset', repo_id='O047/prepSIFT_MathReasoning1'), pr_revision=None, pr_num=None)

In [9]:
df = dfs[2]

print(names[2], df)
df = renameAllColumns(df, ['instruction', 'reasoning_response', 'response'])

print(df)
countNulls(df)
df.push_to_hub("O047/prepSIFT_MathReasoning2", private=False)

GAIR/LIMO DatasetDict({
    train: Dataset({
        features: ['question', 'solution', 'answer'],
        num_rows: 817
    })
})
DatasetDict({
    train: Dataset({
        features: ['instruction', 'reasoning_response', 'response'],
        num_rows: 817
    })
})


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/O047/prepSIFT_MathReasoning2/commit/0f32cd54e4e0dbc9eb7de22c95ba8f450b3236e4', commit_message='Upload dataset', commit_description='', oid='0f32cd54e4e0dbc9eb7de22c95ba8f450b3236e4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/O047/prepSIFT_MathReasoning2', endpoint='https://huggingface.co', repo_type='dataset', repo_id='O047/prepSIFT_MathReasoning2'), pr_revision=None, pr_num=None)

In [10]:
df = dfs[3]

print(names[3], df)

#already well-processed
#filtering for specific language's code snippets, in this case C/CPP for getting language-specific datasets - and possibly augmenting and adding new columns that classify types better for classification tuning
#df.filter(lambda x: print(f"INST:{x['instruction']}\nRESP{x['response']}\n\n\n") if "#include" in x["response"].lower() else None)
countNulls(df)
df.push_to_hub("O047/prepSIFT_Code1", private=False)

ise-uiuc/Magicoder-Evol-Instruct-110K DatasetDict({
    train: Dataset({
        features: ['instruction', 'response'],
        num_rows: 111183
    })
})


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/112 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/O047/prepSIFT_Code1/commit/be2afe1797e771938c3b9cc2efeacd01497d4036', commit_message='Upload dataset', commit_description='', oid='be2afe1797e771938c3b9cc2efeacd01497d4036', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/O047/prepSIFT_Code1', endpoint='https://huggingface.co', repo_type='dataset', repo_id='O047/prepSIFT_Code1'), pr_revision=None, pr_num=None)

In [11]:
df = dfs[4]


print(names[4], df)
df = dropColumns(df, ['ID'])
df = renameAllColumns(df, ['instruction', 'response'])
print(df)
countNulls(df)
df.push_to_hub("O047/prepSIFT_Code2", private=False)

Vezora/Open-Critic-GPT DatasetDict({
    train: Dataset({
        features: ['ID', 'Human', 'Assistant'],
        num_rows: 55117
    })
})
DatasetDict({
    train: Dataset({
        features: ['instruction', 'response'],
        num_rows: 55117
    })
})


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/56 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/O047/prepSIFT_Code2/commit/f9c89b148bdfca162c733692c31dee31bdc5d9f3', commit_message='Upload dataset', commit_description='', oid='f9c89b148bdfca162c733692c31dee31bdc5d9f3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/O047/prepSIFT_Code2', endpoint='https://huggingface.co', repo_type='dataset', repo_id='O047/prepSIFT_Code2'), pr_revision=None, pr_num=None)

In [12]:
df = dfs[5]
print(names[5], dfs[5])
df = dropColumns(df, ['SOURCE'])
df = renameAllColumns(df, ['instruction', 'response'])
print(df)
countNulls(df)
df.push_to_hub("O047/prepSIFT_MathReasoning3", private=False)

qwedsacf/grade-school-math-instructions DatasetDict({
    train: Dataset({
        features: ['INSTRUCTION', 'RESPONSE', 'SOURCE'],
        num_rows: 8792
    })
})
DatasetDict({
    train: Dataset({
        features: ['instruction', 'response'],
        num_rows: 8792
    })
})


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/O047/prepSIFT_MathReasoning3/commit/c577de9c791498b59c7898369cccc52ee7b61e36', commit_message='Upload dataset', commit_description='', oid='c577de9c791498b59c7898369cccc52ee7b61e36', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/O047/prepSIFT_MathReasoning3', endpoint='https://huggingface.co', repo_type='dataset', repo_id='O047/prepSIFT_MathReasoning3'), pr_revision=None, pr_num=None)